In [1]:
from ultralytics import YOLO, settings
import os
import shutil
from IPython.display import display, Image
from IPython import display
import cv2
import matplotlib.pyplot as plt
import numpy as np
from packages import log
display.clear_output()

In [2]:
image = '../../images/test/2234228416.jpg'

In [3]:
MODEL_PATH_OBB = '../..//models/BEST_m_obb.pt'
model_obb = YOLO(MODEL_PATH_OBB)
results = model_obb(image, save=True, conf=0.8)


image 1/1 c:\Users\buyse\Workspace\NTNU\scripts\helper\..\..\images\scale_fixed\2234228416.jpg: 640x416 187.5ms
Speed: 2.0ms preprocess, 187.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 416)
Results saved to C:\Users\buyse\Workspace\NTNU\models\runs\obb\predict


In [3]:
import os

SAVE_DIR_RANDOM = '../../images/cropped_scales/random'

image_files = [f for f in os.listdir(SAVE_DIR_RANDOM) if f.endswith('.jpg') and '_scale_only' not in f and '_grid' not in f]
image_files = [f for f in image_files if not os.path.exists(os.path.join(SAVE_DIR_RANDOM, f.replace('.jpg', '_scale_only.jpg')))]
image_files

['2234228274.jpg', '2461260218.jpg', '2853562301.jpg', '3990750310.jpg']

In [3]:
import os

path = '../../models/runs/detect\\predict\\crops\\scale_fixed'.replace('\\', '/')
if not os.path.exists(path):
    print('Does not exist')

In [2]:
from ultralytics import YOLO, settings
import os
RUNS_DIR = settings['runs_dir']
NO_OBB_PREDICT_PATH = os.path.join(RUNS_DIR, 'detect/predict/crops/scale_fixed')
NO_OBB_PREDICT_PATH

'C:\\Users\\buyse\\Workspace\\NTNU\\models\\runs\\detect/predict/crops/scale_fixed'

In [23]:
from ultralytics import YOLO, settings
import os
import shutil
import cv2
import matplotlib.pyplot as plt

# CONSTANTS
IMG_PATH_FIXED = '../../images/classification/fixed'
MODEL_PATH_NO_OBB = '../../models/BEST_no_obb.pt'
SAVE_DIR_FIXED = '../../images/cropped_scales/fixed'

settings.update({'runs_dir': '../../models/runs'})
RUNS_DIR = settings['runs_dir']

NO_OBB_PREDICT_PATH = os.path.join(RUNS_DIR, 'detect/predict/crops/scale_fixed')

try:
    if not os.path.exists(SAVE_DIR_FIXED):
        os.makedirs(SAVE_DIR_FIXED)
except OSError as e:
    print('Something went wrong, check the log file for more information')

def crop_scale_fixed(images: list):
    model_no_obb = YOLO(MODEL_PATH_NO_OBB)
    # remove old runs
    shutil.rmtree(os.path.join(RUNS_DIR, 'detect'), ignore_errors=True)
    # predict images
    results_fixed = model_no_obb(images, conf=0.8, save_crop=True)

    # move images to save dir
    # predictions = os.listdir(NO_OBB_PREDICT_PATH)
    # for pred in predictions:
    #     if pred.endswith('.jpg'):
    #         shutil.move(os.path.join(NO_OBB_PREDICT_PATH, pred), SAVE_DIR_FIXED)

    return results_fixed


def load_image(image_path, fig_size=(50, 50), grid=False, x_ticks=30, y_ticks=10, x_rotation=0, y_rotation=0, save=False, save_path=None):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image from BGR to RGB (matplotlib uses RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a figure with a specific size
    plt.figure(figsize=fig_size)

    # Show the image
    plt.imshow(image_rgb)

    if grid:
        # Add grid lines to the image for easier measurement
        plt.grid(color='r', linestyle='-', linewidth=0.5)

        # Optionally, you can customize the ticks to match your image's scale
        plt.xticks(range(0, image_rgb.shape[1], x_ticks), rotation=x_rotation)  # Adjust the spacing as needed
        plt.yticks(range(0, image_rgb.shape[0], y_ticks), rotation=y_rotation)  # Adjust the spacing as needed

    if save:
        if save_path is None:
            plt.savefig(image_path.split('/')[-1], bbox_inches='tight', transparent=True)
            plt.close()
        else:
            plt.savefig(save_path, bbox_inches='tight', transparent=True)
            plt.close()

In [24]:
all_images = os.listdir(IMG_PATH_FIXED)
images = [os.path.join(IMG_PATH_FIXED, img).replace('\\', '/') for img in all_images]


In [25]:
images

['../../images/classification/fixed/1701330402.jpg',
 '../../images/classification/fixed/1701302304.jpg',
 '../../images/classification/fixed/1701227243.jpg',
 '../../images/classification/fixed/1701346054.jpg',
 '../../images/classification/fixed/1701771743.jpg']

In [26]:
results = crop_scale_fixed(images)


0: 640x640 1 scale_fixed, 45.0ms
1: 640x640 1 scale_fixed, 45.0ms
2: 640x640 1 scale_fixed, 45.0ms
3: 640x640 1 scale_fixed, 45.0ms
4: 640x640 1 scale_fixed, 45.0ms
Speed: 3.4ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to models/runs/detect/predict


In [16]:
results[0].path.split('/')[-1].replace('.jpg', '_scale_only.jpg')

'1701330402_scale_only.jpg'

In [ ]:
# rename crops
try:
    for idx, result in enumerate(results):
        if idx == 0:
            os.rename(os.path.join(SAVE_DIR_FIXED, 'labels.jpg'), os.path.join(SAVE_DIR_FIXED, result.path.split("/")[-1].replace('.jpg', '') + '_scale_only.jpg'))
        else:
            os.rename(os.path.join(SAVE_DIR_FIXED, f'labels{idx+1}.jpg'), os.path.join(SAVE_DIR_FIXED, result.path.split("/")[-1].replace('.jpg', '') + '_scale_only.jpg'))
except Exception as e:
    log(f'Error renaming crops: {e}')
    print(f'Error renaming crops: {e}')
    print('Something went wrong, check the log file for more information')

In [ ]:


# adds grid to images
try:
    for idx, result in enumerate(results):
        load_image(result.path, grid=True, x_ticks=120, y_ticks=10, x_rotation=90, y_rotation=0, save=True, save_path=os.path.join(SAVE_DIR_FIXED, result.path.split('/')[-1].replace('.jpg', '_grid.jpg')))
except Exception as e:
    log(f'Error adding grid to images: {e}')
    print('Something went wrong, check the log file for more information')

# Move original images to save dir
try:
    for img in images:
        shutil.move(img, SAVE_DIR_FIXED)
except Exception as e:
    log(f'Error moving original images: {e}')
    print('Something went wrong, check the log file for more information')

try:
    for scale in os.listdir(SAVE_DIR_FIXED):
        if scale.endswith('_scale_only.jpg'):
            load_image(os.path.join(SAVE_DIR_FIXED, scale), grid=True, x_ticks=120, y_ticks=10, x_rotation=90, y_rotation=0, save=True, save_path=os.path.join(SAVE_DIR_FIXED, scale.replace('_scale_only.jpg', '_scale_only_grid.jpg')))
except Exception as e:
    log(f'Error adding grid to cropped images: {e}')
    print('Something went wrong, check the log file for more information')

# remove old directories
try:
    shutil.rmtree(IMG_PATH_FIXED)
except Exception as e:
    log(f'Error removing old images: {e}')
    print('Something went wrong, check the log file for more information')